Query rave6 from their website: https://www.rave-survey.org/query/ 

![rave query page](<images/rave-query-page.png>)

#### The query

divide into multiple RAs for smaller response, if needed

```
SELECT TOP 150000 sparv."hrv_sparv" AS rv_rave, sparv."hrv_error_sparv" AS e_rv_rave, aux."teff_sparv" AS teff_rave, aux."logg_sparv" AS logg_rave, aux."m_h_sparv" AS mh_rave, aux."alpha_sparv" AS alphafe_rave, gaia."source_id" 
FROM ravedr6.dr6_sparv AS sparv 
JOIN ravedr6.dr6_sparv_aux AS aux ON sparv.rave_obs_id = aux.rave_obs_id 
RIGHT JOIN ravedr6.dr6_x_gaiaedr3 AS gaia ON gaia.rave_obs_id = sparv.rave_obs_id 
WHERE gaia.ra BETWEEN 0 AND 60
```

Change `WHERE gaia.ra BETWEEN 0 AND 60` into other interval

In [ ]:
# workaround weird error from vaex
# combine all files

import vaex
from glob import glob
from astropy.table import Table

# load all fits files that were downloaded
files = glob("*.fits")
files.sort()

# combine in loop
df_com = []
for file in files:
    table = Table.read(file)
    df_pandas = table.to_pandas()
    df = vaex.from_pandas(df_pandas)
    if len(df_com) == 0:
        df_com = df
    else:
        df_com = df_com.concat(df)
# mask large error
mask = (df_com.e_rv_rave < 200)
df_masked = df_com[mask]


In [ ]:
# remove duplicates, if any
df_pandas = df_masked.to_pandas_df()
df_pandas.drop_duplicates("source_id", inplace=True)
df = vaex.from_pandas(df_pandas)

In [ ]:
# export to hdf5
df.export("rave6.hdf5", progress=True)